In [1]:
from functions import initialize_weaviate_and_get_retriever_with_images

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


In [5]:
retriever = initialize_weaviate_and_get_retriever_with_images(data_folder='./data',collection_name='DocumentosPDFOllama')


✅ Conectado a Weaviate local
🗑️ Colección 'DocumentosPDFOllama' eliminada
📦 Colección 'DocumentosPDFOllama' creada con soporte para imágenes
📄 Procesando diego_velazquez.pdf...
📸 Procesando imagen 1 de la página 1...
✅ Descripción obtenida:  The image is a classic oil painting, depicting a young boy dressed in Renaissance attire atop a hor...
📸 Procesando imagen 2 de la página 1...
✅ Descripción obtenida:  The image depicts a portrait of a man with curly hair, a well-groomed beard and mustache, and a pro...
📸 Procesando imagen 1 de la página 2...
✅ Descripción obtenida:  This image is a captivating oil painting that showcases an intimate scene set in the Renaissance pe...
📸 Procesando imagen 1 de la página 3...
✅ Descripción obtenida:  La imagen es una reproducción de un cuadro o pintura que retrata un ambiente interior con una grupo...
📸 Procesando imagen 2 de la página 3...
✅ Descripción obtenida:  The image is a digital artwork that depicts an outdoor historical scene. It features a

In [ ]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

llm = OllamaLLM(model="qwen3:4b")

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Responde a la pregunta basándote únicamente en el siguiente contexto:\n\n{context}"),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

stuff_documents_chain = create_stuff_documents_chain(llm, answer_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, stuff_documents_chain)

# Ejemplo de uso
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
while True:
    question = input("Escribe tu pregunta (o 'salir' para terminar): ")
    if question.lower() == 'salir':
        break
    
    response = rag_chain.invoke({
        "input": question,
        "chat_history": chat_history
    })
    
    respuesta = response['answer'].split('</think>')[1]
    print(f"Respuesta: {respuesta}\n")
    
    # Actualiza el historial de chat para la siguiente pregunta
    chat_history.extend([HumanMessage(content=question), AIMessage(content=respuesta)])